## Finding the next move

We've built up a useful collection of APL tools. We've got almost all the code we need to implement MENACE.

In this chapter we will add some more functions to our toolbox. One will look at the possible moves from a given board position.
Another will undo a given symmetric transformation.

First, as usual, we will copy in work from the previous chapter and set the index origin to zero.

In [1]:
)copy notebook5

./notebook5.dws saved Mon Dec 14 17:31:48 2020

In [2]:
⎕io ← 0

In [3]:
⍕¨⍳9

┌─┬─┬─┬─┬─┬─┬─┬─┬─┐
│0│1│2│3│4│5│6│7│8│
└─┴─┴─┴─┴─┴─┴─┴─┴─┘

In [4]:
)vars

asymmetric board1 boards encoded hereDir i less mi mn mnp mr180 mr270 mr90 r180 r270 r90 samples start sym symmetric symmetries uc ucp v wpi

In [5]:
samples


0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 1 0 1
0 0 0 0 0 0 1 0 2
0 0 0 0 0 0 2 0 0
0 0 0 0 0 0 2 0 2
2 2 2 2 2 2 2 2 2

In [6]:
list samples

┌───┬───┬───┬───┬───┬───┬───┬───┐
│...│...│...│...│...│...│...│○○○│
│...│...│...│...│...│...│...│○○○│
│...│..×│×..│×.×│×.○│○..│○.○│○○○│
└───┴───┴───┴───┴───┴───┴───┴───┘

In [7]:
⊢board2 ← show samples[4;]

...
...
×.○

In [8]:
⊢empty ← '.'=,board2

1 1 1 1 1 1 0 1 0

In [9]:
3 3⍴↑(1+⍳(+/'.'=,board2)) {⍕¨⍺}@{'.'=⍵} ,board2

123
456
×7○

In [10]:
next_moves ← {3 3⍴↑(1+⍳(+/'.'=,⍵)) {⍕¨⍺}@{'.'=⍵} ,⍵} 

In [11]:
next_moves board2

123
456
×7○

In [12]:
next_moves ¨⊂⍤2show samples

┌───┬───┬───┬───┬───┬───┬───┬───┐
│123│123│123│123│123│123│123│○○○│
│456│456│456│456│456│456│456│○○○│
│789│78×│×78│×7×│×7○│○78│○7○│○○○│
└───┴───┴───┴───┴───┴───┴───┴───┘

In [13]:
⊢n ← 1 0⍕1+⍳9

123456789

In [14]:
next ← {updated ← ,⍵ ⋄ gaps ← '.'=updated ⋄ updated[⍸gaps]← n[⍳+/gaps] ⋄ 3 3⍴updated}

In [15]:
next board2

123
456
×7○

In [16]:
⊂⍤2show samples

┌───┬───┬───┬───┬───┬───┬───┬───┐
│...│...│...│...│...│...│...│○○○│
│...│...│...│...│...│...│...│○○○│
│...│..×│×..│×.×│×.○│○..│○.○│○○○│
└───┴───┴───┴───┴───┴───┴───┴───┘

In [17]:
 ⊢board3 ← samples[7;]

2 2 2 2 2 2 2 2 2

In [18]:
next show board3

○○○
○○○
○○○

In [19]:
next¨⊂⍤2show samples

┌───┬───┬───┬───┬───┬───┬───┬───┐
│123│123│123│123│123│123│123│○○○│
│456│456│456│456│456│456│456│○○○│
│789│78×│×78│×7×│×7○│○78│○7○│○○○│
└───┴───┴───┴───┴───┴───┴───┴───┘

In [20]:
)save notebook6 -force

notebook6.dws saved Wed Dec 16 14:49:11 2020